In [15]:
from IPython.core.debugger import set_trace

In [16]:
import numpy as np
from numpy.random import seed
seed(1)
#display all values from array
np.set_printoptions(threshold=np.nan)
#display not in scientific format
np.set_printoptions(suppress=True)

from tensorflow import set_random_seed
set_random_seed(2)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [17]:
DATA_FREQUENCY = 500
SAMPLING_RATE = 5
FREQUENCY_TO_SAMPLING_RATIO = DATA_FREQUENCY // SAMPLING_RATE

In [ ]:
from data_reader import read_data

(input, target, headers) = read_data(data_path='data', 
                                     sampling_rate=SAMPLING_RATE, 
                                     data_frequency=DATA_FREQUENCY)

Loaded input file: 1
Loaded input file: 2
Loaded input file: 3
Loaded input file: 4
Loaded input file: 5
Loaded input file: 6
Loaded input file: 7
Loaded input file: 8
Loaded input file: 9
Loaded input file: 10
Loaded input file: 11
Loaded input file: 12
Loaded input file: 13
Loaded input file: 14
Loaded input file: 15
Loaded input file: 16
Loaded input file: 17
Loaded input file: 18
Loaded input file: 19
Loaded input file: 20
Loaded input file: 21
Loaded input file: 22
Loaded input file: 23


In [ ]:
def normalize(x):
    scalers = {}
    for channel_number in range(x.shape[1]):
        scalers[channel_number] = StandardScaler()
        x[:, channel_number, :] = scalers[channel_number].fit_transform(x[:, channel_number, :]) 
    return x

In [ ]:
#data without normalization
from chunks_creator import prepare_chunks
from chunks_creator import flatten_chunks

(chunks_input, chunks_target) = prepare_chunks(input, 
                                               target, 
                                               chunk_size_in_seconds=4, 
                                               ratio=FREQUENCY_TO_SAMPLING_RATIO)
(x, y) = flatten_chunks(chunks_input, chunks_target)

x = normalize(x)
    
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

In [ ]:
#callbacks
from keras import callbacks

callbacks_list = [
    callbacks.EarlyStopping(
        monitor='acc', 
        patience=4
    ),
#     callbacks.ModelCheckpoint(
#         filepath='best_model.h5', 
#         monitor='val_loss', 
#         save_best_only=True
#     ),
#     callbacks.TensorBoard(
#         log_dir='logs',
#         histogram_freq=1
#     )
]

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Flatten(input_shape=(400, 16)))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(x_train,
                    y_train,
                    epochs=100,
                    batch_size=64,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list,
                    shuffle=False
#                     validation_split=0.05
                   )

score = model.evaluate(x_test, y_test, batch_size=64)
print("Accuracy: %.2f%%" % (score[1]*100))

In [ ]:
from plots_printer import draw_plots, draw_plots_with_chunks

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 5]

In [ ]:
draw_plots(input, 
           target, 
           headers, 
           patient=0, 
           start_second=900, 
           end_second=910,
           ratio=FREQUENCY_TO_SAMPLING_RATIO)

In [ ]:
draw_plots_with_chunks(input, target, headers, patient=0, chunks_input = chunks_input, to_pdf=False)